In [1]:
from datetime import datetime

from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter as draw
from pytket.utils.operators import QubitPauliOperator
from pytket.partition import measurement_reduction, MeasurementBitMap, MeasurementSetup, PauliPartitionStrat
from pytket.backends.backendresult import BackendResult
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit

from scipy.optimize import minimize
from numpy import ndarray
from numpy.random import random_sample
from sympy import Symbol

import qnexus as qnx
from qnexus.references import CircuitRef

# Example VQE workflow using Quantinuum Nexus


VQE example adapted from https://github.com/CQCL/pytket-quantinuum/blob/develop/examples/Quantinuum_variational_experiment_with_batching.ipynb

## Set up the VQE components

In [2]:
# 1. Synthesise Symbolic State-Preparation Circuit (hardware efficient ansatz)

symbols = [Symbol(f"p{i}") for i in range(4)]
symbolic_circuit = Circuit(2)
symbolic_circuit.X(0)
symbolic_circuit.Ry(symbols[0], 0).Ry(symbols[1], 1)
symbolic_circuit.CX(0, 1)
symbolic_circuit.Ry(symbols[2], 0).Ry(symbols[3], 0)

[X q[0]; Ry(p1) q[1]; Ry(p0) q[0]; CX q[0], q[1]; Ry(p2) q[0]; Ry(p3) q[0]; ]

In [3]:
draw(symbolic_circuit)

In [4]:
# 2. Define Hamiltonian 
# coefficients in the Hamiltonian are obtained from PhysRevX.6.031007

coeffs = [-0.4804, 0.3435, -0.4347, 0.5716, 0.0910, 0.0910]
term0 = {
    QubitPauliString(
        {
            Qubit(0): Pauli.I,
            Qubit(1): Pauli.I,
        }
    ): coeffs[0]
}
term1 = {QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.I}): coeffs[1]}
term2 = {QubitPauliString({Qubit(0): Pauli.I, Qubit(1): Pauli.Z}): coeffs[2]}
term3 = {QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.Z}): coeffs[3]}
term4 = {QubitPauliString({Qubit(0): Pauli.X, Qubit(1): Pauli.X}): coeffs[4]}
term5 = {QubitPauliString({Qubit(0): Pauli.Y, Qubit(1): Pauli.Y}): coeffs[5]}
term_sum = {}
term_sum.update(term0)
term_sum.update(term1)
term_sum.update(term2)
term_sum.update(term3)
term_sum.update(term4)
term_sum.update(term5)
hamiltonian = QubitPauliOperator(term_sum)


In [5]:
# 3 Computing Expectation Values

# Computing Expectation Values for Pauli-Strings
def compute_expectation_paulistring(
    distribution: dict[tuple[int, ...], float], bitmap: MeasurementBitMap
) -> float:
    value = 0
    for bitstring, probability in distribution.items():
        value += probability * (sum(bitstring[i] for i in bitmap.bits) % 2)
    return ((-1) ** bitmap.invert) * (-2 * value + 1)

In [6]:
# 3.2 Computing Expectation Values for sums of Pauli-strings multiplied by coefficients
def compute_expectation_value(
    results: list[BackendResult],
    measurement_setup: MeasurementSetup,
    operator: QubitPauliOperator,
) -> float:
    energy = 0
    for pauli_string, bitmaps in measurement_setup.results.items():
        string_coeff = operator.get(pauli_string, 0.0)
        if string_coeff > 0:
            for bm in bitmaps:
                index = bm.circ_index
                distribution = results[index].get_distribution()
                value = compute_expectation_paulistring(distribution, bm)
                energy += complex(value * string_coeff).real
    return energy

In [8]:
# 4. Building our Objective function

class Objective:
    def __init__(
        self,
        symbolic_circuit: CircuitRef,
        problem_hamiltonian: QubitPauliOperator,
        n_shots_per_circuit: int,
        target: qnx.BackendConfig,
        iteration_number: int = 0,
        n_iterations: int = 10,
    ) -> None:
        """Returns the objective function needed for a variational
        procedure.
        """
        terms = [term for term in problem_hamiltonian._dict.keys()]
        self._symbolic_circuit: Circuit = symbolic_circuit.download_circuit()
        self._hamiltonian: QubitPauliOperator = problem_hamiltonian
        self._nshots: int = n_shots_per_circuit
        self._measurement_setup: MeasurementSetup = measurement_reduction(
            terms, strat=PauliPartitionStrat.CommutingSets
        )
        self._iteration_number: int = iteration_number
        self._niters: int = n_iterations
        self._target = target


    def __call__(self, parameter: ndarray) -> float:
        value = self._objective_function(parameter)
        self._iteration_number += 1
        if self._iteration_number >= self._niters:
            self._iteration_number = 0
        return value
    
    def _objective_function(
        self,
        parameters: ndarray,
    ) -> float:
        
        # Prepare the parameterised state preparation circuit
        assert len(parameters) == len(self._symbolic_circuit.free_symbols())
        symbol_dict = {s: p for s, p in zip(self._symbolic_circuit.free_symbols(), parameters)}
        state_prep_circuit = self._symbolic_circuit.copy()
        state_prep_circuit.symbol_substitution(symbol_dict)

        # Label each job with the properties associated with the circuit.
        properties = {str(sym): val for sym, val in symbol_dict.items()} | {"iteration": self._iteration_number}

        with qnx.context.using_properties(**properties):

            circuit_list = self._build_circuits(state_prep_circuit)

            # Execute circuits with Nexus
            results = qnx.execute(
                name=f"execute_job_VQE_{datetime.now()}_{self._iteration_number}",
                circuits=circuit_list,
                n_shots=[self._nshots]*len(circuit_list),
                backend_config=self._target,
                timeout=None,
            )

        expval = compute_expectation_value(
            results, self._measurement_setup, self._hamiltonian
        )
        return expval

    def _build_circuits(self, state_prep_circuit: Circuit) -> list[CircuitRef]:
        # Requires properties to be set in the context
        
        # Upload the numerical state-prep circuit to Nexus
        qnx.circuit.upload(
            circuit=state_prep_circuit,
            name=f"state prep circuit {self._iteration_number}",
        )
        circuit_list = []
        for mc in self._measurement_setup.measurement_circs:
            c = state_prep_circuit.copy()
            c.append(mc)
            # Upload each measurement circuit to Nexus with correct params
            measurement_circuit_ref = qnx.circuit.upload(
                circuit=c, 
                name=f"state prep circuit {self._iteration_number}",
            )
            circuit_list.append(measurement_circuit_ref)

        # Compile circuits with Nexus
        compiled_circuit_refs = qnx.compile(
            name=f"compile_job_VQE_{datetime.now()}_{self._iteration_number}",
            circuits=circuit_list,
            optimisation_level=2,
            backend_config=self._target,
            timeout=None,
        )
            
        return compiled_circuit_refs

## Set up the Nexus Project and run the VQE

In [9]:
# set up the project
project_ref = qnx.project.create(
    name=f"VQE_example_{str(datetime.now())}",
    description="A VQE done with qnexus",
)

# set this in the context
qnx.context.set_active_project(project_ref)

<Token var=<ContextVar name='qnexus_project' default=None at 0x15feefd30> at 0x305173c40>

### Introducing Properties

Properties are a way to annotate resources in Nexus with custom attributes.

As we will be computing properties in a loop, the iteration number is a natural fit for the property.

In [12]:
qnx.project.add_property(
    name="iteration", 
    property_type="int", 
    description="The iteration number in my dihydrogen VQE experiment", 
)

# Set up the properties for the symbolic circuit parameters
for sym in symbolic_circuit.free_symbols():
    qnx.project.add_property(
        name=str(sym), 
        property_type="float",
        description=f"Our VQE {str(sym)} parameter", 
    )

In [13]:
# Upload our ansatz circuit

ansatz_ref = qnx.circuit.upload(
    circuit=symbolic_circuit,
    name="ansatz_circuit",
    description="The ansatz state-prep circuit for my dihydrogen VQE",
)

## Construct our objective function

In [14]:
objective = Objective(
    symbolic_circuit = ansatz_ref,
    problem_hamiltonian = hamiltonian,
    n_shots_per_circuit = 500,
    n_iterations= 4,
    target = qnx.QuantinuumConfig(device_name="H1-1LE")
)

In [15]:
initial_parameters = random_sample(len(symbolic_circuit.free_symbols()))

result = minimize(
    objective,
    initial_parameters,
    method="COBYLA",
    options={"disp": True, "maxiter": objective._niters},
    tol=1e-2,
)

print(result.fun)
print(result.x)

-0.8997872
   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =    4   F =-8.997872E-01    MAXCV = 0.000000E+00
   X = 1.806205E+00   9.958740E-01   9.968749E-02   1.099286E-01

[1.80620461 0.995874   0.09968749 0.10992865]


# Use Nexus to Rescue a VQE workflow

For instance, lets say that some failure happened on the 2nd iteration (e.g. laptop ran out of battery) and we want to resume ASAP.

In the above we ran for 4 iterations, lets pretend that we actually wanted to run for 7 and it failed on the 4th one.

N.B. Ignoring any minimizer internal state.

In [16]:
# Get the project, fetching the latest one with the name prefix from above
project_matches = qnx.project.get(name_like="VQE_example_", sort_filters=['-created'])

project_ref = project_matches.list()[0]

# set this in the context
qnx.context.set_active_project(project_ref)

project_ref.df()

,name,description,created,modified,contents_modified,id
0,VQE_example_2024-07-12 10:58:32.959202,A VQE done with qnexus,2024-07-12 09:58:34.341059+00:00,2024-07-12 09:58:34.341059+00:00,2024-07-12 10:02:52.883085+00:00,ddd20b97-97b3-454d-9526-78f6c5a3a5c9


In [17]:
# Get the symbolic circuit
symbolic_circuit_ref = qnx.circuit.get_only(name_like="ansatz_circuit")

In [18]:
most_recent_circuits = qnx.circuit.get(name_like="final", project=project_ref)

most_recent_circuits.summarize()

,resource,total_count
0,Circuit,8


In [19]:
most_recent_circuits_refs = most_recent_circuits.list()

most_recent_circuits_refs.df()

,name,description,created,modified,iteration,p0,p1,p2,p3,project,id
0,state prep circuit 0-QuantinuumBackend-final,None,2024-07-12 09:59:24.550799+00:00,2024-07-12 09:59:24.550799+00:00,0,0.109929,0.995874,0.099687,0.806205,VQE_example_2024-07-12 10:58:32.959202,7df46ff1-7668-400e-ac0e-0b1542e6f60d
1,state prep circuit 0-QuantinuumBackend-final,None,2024-07-12 09:59:25.100608+00:00,2024-07-12 09:59:25.100608+00:00,0,0.109929,0.995874,0.099687,0.806205,VQE_example_2024-07-12 10:58:32.959202,b11bdd0f-3fa5-4561-b093-1daec328cbc6
2,state prep circuit 1-QuantinuumBackend-final,None,2024-07-12 10:01:52.133075+00:00,2024-07-12 10:01:52.133075+00:00,1,0.109929,0.995874,0.099687,1.806205,VQE_example_2024-07-12 10:58:32.959202,1b9a1226-aad4-48c4-ad3d-f98494d26700
3,state prep circuit 1-QuantinuumBackend-final,None,2024-07-12 10:01:52.523121+00:00,2024-07-12 10:01:52.523121+00:00,1,0.109929,0.995874,0.099687,1.806205,VQE_example_2024-07-12 10:58:32.959202,ea95d515-0a78-467c-aba2-8713aa1a7c6a
4,state prep circuit 2-QuantinuumBackend-final,None,2024-07-12 10:02:14.662651+00:00,2024-07-12 10:02:14.662651+00:00,2,0.109929,1.995874,0.099687,1.806205,VQE_example_2024-07-12 10:58:32.959202,33be19e9-8f0a-4460-806a-813beedce49d
5,state prep circuit 2-QuantinuumBackend-final,None,2024-07-12 10:02:15.136188+00:00,2024-07-12 10:02:15.136188+00:00,2,0.109929,1.995874,0.099687,1.806205,VQE_example_2024-07-12 10:58:32.959202,91588720-1580-4435-b465-09cda2f1a426
6,state prep circuit 3-QuantinuumBackend-final,None,2024-07-12 10:02:37.235612+00:00,2024-07-12 10:02:37.235612+00:00,3,0.109929,0.995874,1.099687,1.806205,VQE_example_2024-07-12 10:58:32.959202,485f642f-5a04-4b1f-b469-cff8e2755ebb
7,state prep circuit 3-QuantinuumBackend-final,None,2024-07-12 10:02:37.656389+00:00,2024-07-12 10:02:37.656389+00:00,3,0.109929,0.995874,1.099687,1.806205,VQE_example_2024-07-12 10:58:32.959202,a76feddb-c091-44ff-ba66-04863378e9d5


In [20]:
# Get the latest circuit to get the new 'initial_parameters'
latest_circuit: CircuitRef = most_recent_circuits_refs[-1]

latest_circuit_properties = latest_circuit.annotations.properties

latest_circuit.df()

,name,description,created,modified,iteration,p0,p1,p2,p3,project,id
0,state prep circuit 3-QuantinuumBackend-final,None,2024-07-12 10:02:37.656389+00:00,2024-07-12 10:02:37.656389+00:00,3,0.109929,0.995874,1.099687,1.806205,VQE_example_2024-07-12 10:58:32.959202,a76feddb-c091-44ff-ba66-04863378e9d5


In [21]:
# Get what iteration we were on (from the latest circuit)

last_iteration_count = latest_circuit_properties.pop("iteration")

print(last_iteration_count)

# Retreive the params and check them
new_starting_params = list(latest_circuit_properties.values())
print(new_starting_params)


3
[0.10992864519357681, 0.9958739876747131, 1.0996874570846558, 1.8062045574188232]


In [22]:
# Build the Objective and run 'minimize' to continue the experiment
objective = Objective(
    symbolic_circuit_ref,
    hamiltonian,
    n_shots_per_circuit = 500,
    iteration_number=last_iteration_count, # resume from 3rd iteration of 7
    n_iterations = 7,
    target = qnx.QuantinuumConfig(device_name="H1-1LE")
)

result = minimize(
    objective,
    new_starting_params,
    method="COBYLA",
    options={"disp": True, "maxiter": objective._niters},
    tol=1e-2,
)

print(result.fun)
print(result.x)

-0.5878891999999998
   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =    7   F =-5.878892E-01    MAXCV = 0.000000E+00
   X = 1.109929E+00   9.958740E-01   1.099687E+00   1.806205E+00

[1.10992865 0.99587399 1.09968746 1.80620456]


### Querying by property

In [2]:
qnx.circuit.get(properties={"iteration": 5}).df()

,name,description,created,modified,iteration,p0,p1,p2,p3,project,id
0,state prep circuit 5,A numerical state-prep circuit for my dihydrog...,2024-02-20 10:59:00.939749+00:00,2024-02-20 10:59:00.939749+00:00,5,1.838456,0.773245,2.066804,0.825692,VQE_example_2024-02-20 10:45:09.706117,518662d6-7e6a-4f0a-b4f1-c40b037c8852
1,state prep circuit 5,A numerical state-prep circuit for my dihydrog...,2024-02-20 10:59:01.163727+00:00,2024-02-20 10:59:01.163727+00:00,5,1.838456,0.773245,2.066804,0.825692,VQE_example_2024-02-20 10:45:09.706117,b07ffbb1-e9b9-43bd-a559-366ddf1316e7
2,state prep circuit 5,A numerical state-prep circuit for my dihydrog...,2024-02-20 10:59:01.377086+00:00,2024-02-20 10:59:01.377086+00:00,5,1.838456,0.773245,2.066804,0.825692,VQE_example_2024-02-20 10:45:09.706117,c88b2907-0b1c-4bf7-8018-a66cf2c420fb
3,state prep circuit 5-AerBackend-final,None,2024-02-20 10:59:02.653149+00:00,2024-02-20 10:59:02.653149+00:00,5,1.838456,0.773245,2.066804,0.825692,VQE_example_2024-02-20 10:45:09.706117,86e7297c-7732-49c8-84be-af45bb6daf58
4,state prep circuit 5-AerBackend-final,None,2024-02-20 10:59:02.922675+00:00,2024-02-20 10:59:02.922675+00:00,5,1.838456,0.773245,2.066804,0.825692,VQE_example_2024-02-20 10:45:09.706117,4fadf49d-b7e6-4649-b310-11956790ff4a
5,state prep circuit 5,A numerical state-prep circuit for my dihydrog...,2024-03-21 13:45:54.761009+00:00,2024-03-21 13:45:54.761009+00:00,5,2.933704,1.646044,0.801043,0.533589,VQE_example_2024-01-11 16:44:56.065265,b4507d8a-d690-47dc-b999-a823fde6b206
6,state prep circuit 5,A numerical state-prep circuit for my dihydrog...,2024-03-21 13:45:55.005858+00:00,2024-03-21 13:45:55.005858+00:00,5,2.933704,1.646044,0.801043,0.533589,VQE_example_2024-01-11 16:44:56.065265,87a8e437-8cdf-40d1-af5b-59665134440e
7,state prep circuit 5,A numerical state-prep circuit for my dihydrog...,2024-03-21 13:45:55.222357+00:00,2024-03-21 13:45:55.222357+00:00,5,2.933704,1.646044,0.801043,0.533589,VQE_example_2024-01-11 16:44:56.065265,2a5e79d3-27e4-4fef-9d31-866ed7148dd9
8,state prep circuit 5-AerBackend-final,None,2024-03-21 13:45:56.414603+00:00,2024-03-21 13:45:56.414603+00:00,5,2.933704,1.646044,0.801043,0.533589,VQE_example_2024-01-11 16:44:56.065265,ae49678d-b621-46cc-99d8-5e37b55d723a
9,state prep circuit 5-AerBackend-final,None,2024-03-21 13:45:56.657348+00:00,2024-03-21 13:45:56.657348+00:00,5,2.933704,1.646044,0.801043,0.533589,VQE_example_2024-01-11 16:44:56.065265,b90bcac9-195a-4241-a5a4-43553bbcdd6c
